In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets,layers,optimizers,Sequential, metrics

In [10]:
# fashion Mnist 实战

def preprocess(x,y):
    x=tf.cast(x,dtype=tf.float32)/255.
    y=tf.cast(y,dtype=tf.int32)
    
    return x,y

(x,y),(x_test,y_test)=datasets.fashion_mnist.load_data()
print(x.shape,y.shape)

batchsz=128
db=tf.data.Dataset.from_tensor_slices((x,y))
db=db.map(preprocess).shuffle(10000).batch(batchsz)

db_test=tf.data.Dataset.from_tensor_slices((x_test,y_test))
db_test=db_test.map(preprocess).batch(batchsz)

db_iter=iter(db)
sample=next(db_iter)
print('sample:',sample[0].shape,sample[1].shape)

model=Sequential([
    layers.Dense(256,activation=tf.nn.relu), # [b,784]->[b,256]
    layers.Dense(128,activation=tf.nn.relu), # [b,256]->[b,128]
    layers.Dense(64,activation=tf.nn.relu), # [b,128]->[b,64]
    layers.Dense(32,activation=tf.nn.relu), # [b,64->[b,32]
    layers.Dense(10) # [b,32]->[b,10]
])

model.build(input_shape=[None,28*28])
model.summary()

# w=w-lr*grad
optimizer=optimizers.Adam(lr=1e-3)

def main():
    
    for epoch in range(30):
        
        for step, (x,y) in enumerate(db):
            
            # x: [b,28,28]->[b,28*28]
            # y:[b]
            x=tf.reshape(x,[-1,28*28])
            
            with tf.GradientTape() as tape:
                #[b,784] -> [b,10]
                logits=model(x)
                y_onehot=tf.one_hot(y,depth=10)
                loss_mse=tf.reduce_mean(tf.losses.MSE(y_onehot,logits))
                loss_entropy=tf.reduce_mean(tf.losses.categorical_crossentropy(y_onehot,logits,from_logits=True))
                
            grads=tape.gradient(loss_entropy,model.trainable_variables)
            optimizer.apply_gradients(zip(grads,model.trainable_variables))
            
            if step % 100 ==0:
                print(epoch, step, 'loss:',float(loss_entropy), float(loss_mse))
                
        # 测试
        total_correct=0
        total_num=0
        for x,y in db_test:
            # x: [b,28,28]->[b,28*28]
            # y:[b]
            x=tf.reshape(x,[-1,28*28])
            logits=model(x)
            # logits -> prob, [b,10]
            prob=tf.nn.softmax(logits,axis=1)
            #[b,10]->[b], int64
            pred=tf.argmax(prob,axis=1)
            pred=tf.cast(pred,dtype=tf.int32)
            # correct: [b], true, false,false, 
            correct=tf.equal(pred,y)
            correct=tf.reduce_sum(tf.cast(correct,dtype=tf.int32))
            total_correct=total_correct+int(correct)
            total_num=total_num+x.shape[0]
        
        acc=total_correct/total_num
        print(epoch,'test acc:',acc)
            
                
main()
            

(60000, 28, 28) (60000,)
sample: (128, 28, 28) (128,)
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             multiple                  200960    
_________________________________________________________________
dense_31 (Dense)             multiple                  32896     
_________________________________________________________________
dense_32 (Dense)             multiple                  8256      
_________________________________________________________________
dense_33 (Dense)             multiple                  2080      
_________________________________________________________________
dense_34 (Dense)             multiple                  330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________
0 0 loss: 2.340846538543701 0.14102143049240112
0 100 loss: 

KeyboardInterrupt: 